# Ensemble Learning
In statistics and machine learning, **ensemble methods** use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms alone. Simply put,

1. Ensemble models in machine learning combine the decisions from multiple models to improve the overall performance.


2. The main causes of error in learning models are due to noise, bias and variance. Ensemble methods help to minimize these factors.


3. Ensembles are a divide and conquer approach used to improve performance.

Ensemble learning can help you to build a really robust model from a few "weak" models, which eliminates a lot of the model tuning that would else be needed to achieve good results. They are especially popular in data science competitions because for these competitions the highest accuracy is more important than the runtime or interpretability of the model.

In [1]:
import pathlib
import numpy as np
import pandas as pd
from scipy.stats import norm, skew
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score

# House Prices

In this notebook, we will go through the most common ensembling techniques out there. We will work on the [House Prices Regression dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview) which can be freely downloaded from Kaggle.

In [2]:
path = pathlib.Path("datasets")

In [3]:
# Load in the train and test dataset
train = pd.read_csv(path/'train.csv')
test = pd.read_csv(path/'test.csv')

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train.isnull().sum().sort_values(ascending=False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
LotFrontage       259
GarageCond         81
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageQual         81
BsmtExposure       38
BsmtFinType2       38
BsmtFinType1       37
BsmtCond           37
BsmtQual           37
MasVnrArea          8
MasVnrType          8
Electrical          1
Utilities           0
YearRemodAdd        0
MSSubClass          0
Foundation          0
ExterCond           0
ExterQual           0
Exterior2nd         0
Exterior1st         0
RoofMatl            0
RoofStyle           0
YearBuilt           0
                 ... 
GarageArea          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
3SsnPorch           0
BsmtUnfSF           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
Functional          0
TotRmsAbvGrd        0
KitchenQua

In [5]:
# Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

# Drop id column because is is unnecessary for making predictions
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)